In [57]:
import pandas as pd
import sqlite3 
import plotly.graph_objects as go
import random
import matplotlib.colors as mcolors

In [4]:
# Get all columns
pd.set_option('display.max_columns', None)

# Remove float calculations
pd.set_option('display.float_format', lambda x: '%.3f' % x)

For understanding the data use this site:
https://www.baseball-almanac.com/stats4.shtml

##### Connect to sqlite file, setting up tables, combining like tables

In [5]:
conn = sqlite3.connect('baseball_data\database.sqlite')

In [6]:
get_tables = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

cursor = conn.cursor()
cursor.execute(get_tables)

sql_names = []
for name in cursor.fetchall():
    sql_names.append(name[0])

In [7]:
list_df = []
for name in sql_names:
    df = pd.read_sql_query(f"SELECT * FROM {name}", conn)
    list_df.append(df)

Assigning variables to each dataframe in list

In [8]:
all_star = list_df[0]
appearances = list_df[1]
manager_award = list_df[2]
player_award = list_df[3]
manager_award_vote = list_df[4]
player_award_vote = list_df[5]
batting = list_df[6]
batting_postseason = list_df[7]
player_college = list_df[8]
fielding = list_df[9]
fielding_outfield = list_df[10]
fielding_postseason = list_df[11]
hall_of_fame = list_df[12]
home_game = list_df[13]
manager = list_df[14]
manager_half = list_df[15]
player = list_df[16]
park = list_df[17]
pitching = list_df[18]
pitching_postseason = list_df[19]
college = list_df[21]
postseason = list_df[22]
team = list_df[23]
team_francise = list_df[24]
team_half = list_df[25]

#Part of Player
salary = list_df[20]

In [9]:
all_star['game_num'].value_counts()

game_num
0    4615
2     240
1     214
Name: count, dtype: int64

,year,league_id,team_id,franchise_id,div_id,rank,g,ghome,w,l,div_win,wc_win,lg_win,ws_win,r,ab,h,double,triple,hr,bb,so,sb,cs,hbp,sf,ra,er,era,cg,sho,sv,ipouts,ha,hra,bba,soa,e,dp,fp,name,park,attendance,bpf,ppf,team_id_br,team_id_lahman45,team_id_retro
0,1871,,BS1,BNA,,3,31,,20,10,,,N,,401,1372,426,70,37,3,60,19,73,,,,303,109,3.550,22,1,3,828,367,2,42,23,225,,0.830,Boston Red Stockings,South End Grounds I,,103,98,BOS,BS1,BS1
1,1871,,CH1,CNA,,2,28,,19,9,,,N,,302,1196,323,52,21,10,60,22,69,,,,241,77,2.760,25,0,1,753,308,6,28,22,218,,0.820,Chicago White Stockings,Union Base-Ball Grounds,,104,102,CHI,CH1,CH1
2,1871,,CL1,CFC,,8,29,,10,19,,,N,,249,1186,328,35,40,7,26,25,18,,,,341,116,4.110,23,0,0,762,346,13,53,34,223,,0.810,Cleveland Forest Citys,National Association Grounds,,96,100,CLE,CL1,CL1
3,1871,,FW1,KEK,,7,19,,7,12,,,N,,137,746,178,19,8,2,33,9,16,,,,243,97,5.170,19,1,0,507,261,5,21,17,163,,0.800,Fort Wayne Kekiongas,Hamilton Field,,101,107,KEK,FW1,FW1
4,1871,,NY2,NNA,,5,33,,16,17,,,N,,302,1404,403,43,21,1,33,15,46,,,,313,121,3.720,32,1,0,879,373,7,42,22,227,,0.830,New York Mutuals,Union Grounds (Brooklyn),,90,88,NYU,NY2,NY2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2800,2015,NL,LAN,LAD,W,1,162,81,92,70,Y,N,N,N,667,5385,1346,263,26,187,563,1258,59,34,60,30,595,553,3.440,6,21,47,4337,1317,145,395,1396,75,133,0.988,Los Angeles Dodgers,Dodger Stadium,3764815,101,98,LAD,LAN,LAN
2801,2015,NL,SFN,SFG,W,2,162,81,84,78,N,N,N,N,696,5565,1486,288,39,136,457,1159,93,36,49,37,627,597,3.720,7,18,41,4333,1344,155,431,1165,78,145,0.987,San Francisco Giants,AT&T Park,3375882,99,97,SFG,SFN,SFN
2802,2015,NL,ARI,ARI,W,3,162,81,79,83,N,N,N,N,720,5649,1494,289,48,154,490,1312,132,44,33,57,713,659,4.040,1,12,44,4400,1450,182,500,1215,86,146,0.986,Arizona Diamondbacks,Chase Field,2080145,107,106,ARI,ARI,ARI
2803,2015,NL,SDN,SDP,W,4,162,81,74,88,N,N,N,N,650,5457,1324,260,36,148,426,1327,82,29,40,42,731,655,4.090,1,6,41,4321,1371,171,516,1393,92,138,0.985,San Diego Padres,Petco Park,2459742,98,97,SDP,SDN,SDN


Combining Salary and Player

In [10]:
salary_new = salary.drop_duplicates(subset=['player_id'], keep='last')
salary_new = salary[['player_id', 'team_id', 'salary']]

player = player.merge(salary_new, on='player_id', how='left')

##### Query 1: Average salary per team

In [11]:
player.sort_values(by=['salary'], ascending=False)[['player_id', 'name_first', 'name_last', 'name_given', 'team_id', 'salary']]

,player_id,name_first,name_last,name_given,team_id,salary
30351,rodrial01,Alex,Rodriguez,Alexander Enmanuel,NYA,33000000.000
30352,rodrial01,Alex,Rodriguez,Alexander Enmanuel,NYA,33000000.000
18735,kershcl01,Clayton,Kershaw,Clayton Edward,LAN,32571000.000
30353,rodrial01,Alex,Rodriguez,Alexander Enmanuel,NYA,32000000.000
30354,rodrial01,Alex,Rodriguez,Alexander Enmanuel,NYA,30000000.000
...,...,...,...,...,...,...
39436,zuberbi01,Bill,Zuber,William Henry,NaN,NaN
39450,zupofr01,Frank,Zupo,Frank Joseph,NaN,NaN
39452,zuverge01,George,Zuverink,George,NaN,NaN
39453,zwilldu01,Dutch,Zwilling,Edward Harrison,NaN,NaN


In [277]:
#Get average salary per team based on players
avg_salary = player.groupby(by='team_id')[['salary']].mean().reset_index()

#Get full team name
avg_salary = avg_salary.merge(team[['team_id', 'name']], on='team_id', how='inner')


In [279]:
# Get rid of teams that have had name changes, keep the most recent name change
avg_salary = avg_salary.drop_duplicates(subset=['team_id', 'salary'], keep='last')[['name', 'team_id', 'salary']]

avg_salary.sort_values(by=['salary'], ascending=False)

,name,team_id,salary
1053,Los Angeles Angels of Anaheim,LAA,4208635.810
1365,New York Yankees,NYA,3850078.514
252,Boston Red Sox,BOS,2849574.924
2117,Washington Nationals,WAS,2730242.925
1111,Los Angeles Dodgers,LAN,2660758.635
1115,Miami Marlins,MIA,2555181.188
25,Arizona Diamondbacks,ARI,2483006.305
1419,New York Mets,NYN,2387047.242
1873,San Francisco Giants,SFN,2290805.243
1600,Philadelphia Phillies,PHI,2278616.757


##### Query 2: Players who then became managers

In [14]:
# One manager can be on different teams during their career 
manager.sort_values(by=['player_id'])

,player_id,year,team_id,league_id,inseason,g,w,l,rank,plyr_mgr
3206,actama99,2010,CLE,AL,1,162,69,93,4,N
3199,actama99,2009,WAS,NL,1,87,26,61,5,N
3242,actama99,2011,CLE,AL,1,162,80,82,2,N
3166,actama99,2008,WAS,NL,1,161,59,102,5,N
3132,actama99,2007,WAS,NL,1,162,73,89,4,N
...,...,...,...,...,...,...,...,...,...,...
2297,zimmedo01,1982,TEX,AL,1,96,38,58,6,N
2110,zimmedo01,1977,BOS,AL,1,161,97,64,3,N
2148,zimmedo01,1978,BOS,AL,1,163,99,64,2,N
2018,zimmedo01,1973,SDN,NL,1,162,60,102,6,N


In [15]:
# Join manager table and player table on the player_ID
player_to_manager = manager[['player_id']].merge(player[['player_id', 'name_first', 'name_last', 'name_given']], on='player_id', how='inner')

In [16]:
# Get unique managers, to get an accurate list of managers who used to be players
player_to_manager = player_to_manager.drop_duplicates()
player_to_manager.sort_values(by=['player_id'])

,player_id,name_first,name_last,name_given
4389,actama99,Manny,Acta,Manuel Elias
2097,adairbi99,Bill,Adair,Marion Danne
1993,adcocjo01,Joe,Adcock,Joseph Wilbur
128,addybo01,Bob,Addy,Robert Edward
451,allenbo01,Bob,Allen,Robert Gilman
...,...,...,...,...
3872,yostne01,Ned,Yost,Edgar Frederick
819,youngcy01,Cy,Young,Denton True
65,youngni99,Nick,Young,Nicholas Ephraim
771,zimmech01,Chief,Zimmer,Charles Louis


##### Query 3: Get total number of hits, double, triples, walks, and homeruns per team per year and how much that team made

In [17]:
team_year_stats = team[['year', 'name', 'team_id', 'h', 'double', 'triple', 'bb', 'hr']]
team_year_stats

,year,name,team_id,h,double,triple,bb,hr
0,1871,Boston Red Stockings,BS1,426,70,37,60,3
1,1871,Chicago White Stockings,CH1,323,52,21,60,10
2,1871,Cleveland Forest Citys,CL1,328,35,40,26,7
3,1871,Fort Wayne Kekiongas,FW1,178,19,8,33,2
4,1871,New York Mutuals,NY2,403,43,21,33,1
...,...,...,...,...,...,...,...,...
2800,2015,Los Angeles Dodgers,LAN,1346,263,26,563,187
2801,2015,San Francisco Giants,SFN,1486,288,39,457,136
2802,2015,Arizona Diamondbacks,ARI,1494,289,48,490,154
2803,2015,San Diego Padres,SDN,1324,260,36,426,148


In [18]:
salary_team_q3 = salary[['year', 'team_id', 'salary']]
salary_team_q3 = salary_team_q3.groupby(by=['year', 'team_id']).sum().reset_index()
salary_team_q3

,year,team_id,salary
0,1985,ATL,14807000
1,1985,BAL,11560712
2,1985,BOS,10897560
3,1985,CAL,14427894
4,1985,CHA,9846178
...,...,...,...
883,2015,SLN,119241500
884,2015,TBA,64521233
885,2015,TEX,143742789
886,2015,TOR,112992400


In [19]:
final_q3 = team_year_stats.merge(salary_team_q3, how='inner', on=['year', 'team_id'])
final_q3 = final_q3.rename(columns={'salary' : 'money_made'})

final_q3

,year,name,team_id,h,double,triple,bb,hr,money_made
0,1985,Atlanta Braves,ATL,1359,213,28,553,126,14807000
1,1985,Baltimore Orioles,BAL,1451,234,22,604,214,11560712
2,1985,Boston Red Sox,BOS,1615,292,31,562,162,10897560
3,1985,California Angels,CAL,1364,215,31,648,153,14427894
4,1985,Chicago White Sox,CHA,1386,247,37,471,146,9846178
...,...,...,...,...,...,...,...,...,...
883,2015,Los Angeles Dodgers,LAN,1346,263,26,563,187,215792000
884,2015,San Francisco Giants,SFN,1486,288,39,457,136,164701500
885,2015,Arizona Diamondbacks,ARI,1494,289,48,490,154,61834000
886,2015,San Diego Padres,SDN,1324,260,36,426,148,118441300


In [244]:
batting

,player_id,year,stint,team_id,league_id,g,ab,r,h,double,triple,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,g_idp
0,abercda01,1871,1,TRO,,1,4,0,0,0,0,0,0,0,0,0,0,,,,,
1,addybo01,1871,1,RC1,,25,118,30,32,6,0,0,13,8,1,4,0,,,,,
2,allisar01,1871,1,CL1,,29,137,28,40,4,5,0,19,3,1,2,5,,,,,
3,allisdo01,1871,1,WS3,,27,133,28,44,10,2,2,27,1,1,0,2,,,,,
4,ansonca01,1871,1,RC1,,25,120,29,39,11,3,0,16,6,2,2,1,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101327,zitoba01,2015,1,OAK,AL,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
101328,zobribe01,2015,1,OAK,AL,67,235,39,63,20,2,6,33,1,1,33,26,2,0,0,3,5
101329,zobribe01,2015,2,KCA,AL,59,232,37,66,16,1,7,23,2,3,29,30,1,1,0,2,3
101330,zuninmi01,2015,1,SEA,AL,112,350,28,61,11,0,11,28,0,1,21,132,0,5,8,2,6


##### Visualization: Player Batting Career Stats

In [245]:
# g - Games
# ab - At Bats
# r - Runs
# h - Hits
# double
# triple
# hr - home runs
# rbi - Runs Batted In
# sb - stolen bases
# cs - caught stealing

# Setting up specific columns to showcase

player_batting = batting[['player_id','ab', 'r', 'h', 'double', 'triple', 'hr', 'bb']]
player_batting = player_batting.rename(columns={'ab' : 'At Bat', 'r' : 'Runs', 'h' : 'Hits', 'double' : 'Double', 'triple' : 'Triple', 'hr' : 'Home Runs', 'bb' : 'Walks'})
player_batting

,player_id,At Bat,Runs,Hits,Double,Triple,Home Runs,Walks
0,abercda01,4,0,0,0,0,0,0
1,addybo01,118,30,32,6,0,0,4
2,allisar01,137,28,40,4,5,0,2
3,allisdo01,133,28,44,10,2,2,0
4,ansonca01,120,29,39,11,3,0,2
...,...,...,...,...,...,...,...,...
101327,zitoba01,0,0,0,0,0,0,0
101328,zobribe01,235,39,63,20,2,6,33
101329,zobribe01,232,37,66,16,1,7,29
101330,zuninmi01,350,28,61,11,0,11,21


In [246]:
# Turning strings into ints
player_batting = player_batting.loc[player_batting['At Bat'] != '']
player_batting = player_batting.astype({'At Bat':'int', 'Runs':'int', 'Hits':'int', 'Double':'int', 'Triple':'int', 'Home Runs':'int', 'Walks':'int'})

# Getting total career batting stats

player_batting_career = player_batting.groupby(by=['player_id']).sum().reset_index()

player_batting_career = player[['player_id', 'name_given']].drop_duplicates().merge(player_batting_career, on='player_id')
player_batting_career

,player_id,name_given,At Bat,Runs,Hits,Double,Triple,Home Runs,Walks
0,aardsda01,David Allan,4,0,0,0,0,0,0
1,aaronha01,Henry Louis,12364,2174,3771,624,98,755,1402
2,aaronto01,Tommie Lee,944,102,216,42,6,13,86
3,aasedo01,Donald William,5,0,0,0,0,0,0
4,abadan01,Fausto Andres,21,1,2,0,0,0,4
...,...,...,...,...,...,...,...,...,...
18227,zupofr01,Frank Joseph,18,3,3,1,0,0,2
18228,zuvelpa01,Paul,491,41,109,17,2,2,34
18229,zuverge01,George,142,5,21,2,1,0,9
18230,zwilldu01,Edward Harrison,1280,167,364,76,15,30,128


In [348]:
def random_color():
    return random.choice(list(mcolors.CSS4_COLORS))

fig = go.Figure()
fig.add_trace(
    go.Scatterpolar(
        mode='markers',
        theta=['Walks', 'Double', 'Triple', 'Home Runs'],
        fill='toself',
        marker_color='black'
    )
)

data = player_batting_career.loc[player_batting_career['player_id'] == player_batting_career.sample()[['player_id']].values[0][0]]
totalVal = sum(data[['Walks', 'Double', 'Triple', 'Home Runs']].values.flatten().tolist())
while (totalVal <= 20):
    data = player_batting_career.loc[player_batting_career['player_id'] == player_batting_career.sample()[['player_id']].values[0][0]]
    totalVal = sum(data[['Walks', 'Double', 'Triple', 'Home Runs']].values.flatten().tolist())

title = f"{data[['player_id']].values[0][0]} - {data[['name_given']].values[0][0]}"
fig.update_traces(
    r = data[['Walks', 'Double', 'Triple', 'Home Runs']].values.flatten().tolist(),
    name = title,
    fillcolor=random_color()
)

fig.update_layout(
    title = {
        'text' : title,
        'x' : 0.5
    }
)
fig.show()